## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.insert(0, '..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hambantota,6.1241,81.1185,overcast clouds,79.70,79,100,19.19,LK,2021-08-02 16:16:34
1,1,Ribeira Grande,38.5167,-28.7000,scattered clouds,77.40,78,40,8.05,PT,2021-08-02 16:16:34
2,2,Fairbanks,64.8378,-147.7164,clear sky,70.92,73,1,3.44,US,2021-08-02 16:09:53
3,3,Hobart,-42.8794,147.3294,overcast clouds,48.25,78,100,5.01,AU,2021-08-02 16:07:43
4,4,Severo-Kurilsk,50.6789,156.1250,overcast clouds,51.76,88,87,18.70,RU,2021-08-02 16:09:22


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for you trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for you trip?90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,City_ID,City,Lat,Lng,Weather Description,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Hambantota,6.1241,81.1185,overcast clouds,79.70,79,100,19.19,LK,2021-08-02 16:16:34
1,1,Ribeira Grande,38.5167,-28.7000,scattered clouds,77.40,78,40,8.05,PT,2021-08-02 16:16:34
7,7,Ixtapa,20.7000,-105.2000,broken clouds,85.23,78,75,6.91,MX,2021-08-02 16:15:59
12,12,Menongue,-14.6585,17.6910,few clouds,78.75,15,19,4.76,AO,2021-08-02 16:16:37
14,14,Komsomolskiy,40.4272,71.7189,overcast clouds,83.12,32,100,2.30,UZ,2021-08-02 16:12:50


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().count()

City_ID                276
City                   276
Lat                    276
Lng                    276
Weather Description    276
Max Temp               276
Humidity               276
Cloudiness             276
Wind Speed             276
Country                276
Date                   276
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Lat                    0
Lng                    0
Weather Description    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Date                   0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Weather Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Hambantota,LK,79.70,overcast clouds,6.1241,81.1185,
1,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,
7,Ixtapa,MX,85.23,broken clouds,20.7000,-105.2000,
12,Menongue,AO,78.75,few clouds,-14.6585,17.6910,
14,Komsomolskiy,UZ,83.12,overcast clouds,40.4272,71.7189,
16,Bathsheba,BB,86.61,few clouds,13.2167,-59.5167,
19,San Eduardo,PH,82.33,overcast clouds,12.1679,125.4503,
22,Avarua,CK,78.85,light rain,-21.2078,-159.7750,
30,Butaritari,KI,82.63,broken clouds,3.0707,172.7902,
31,Sacramento,US,85.71,broken clouds,38.4666,-121.3177,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found. Skipping...")
hotel_df.head(10)

Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...
Hotel not found. Skipping...


,City,Country,Max Temp,Weather Description,Lat,Lng,Hotel Name
0,Hambantota,LK,79.70,overcast clouds,6.1241,81.1185,Bungalow 63
1,Ribeira Grande,PT,77.40,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
7,Ixtapa,MX,85.23,broken clouds,20.7000,-105.2000,Hotel Plaza Providencia
12,Menongue,AO,78.75,few clouds,-14.6585,17.6910,Ritz Lauca
14,Komsomolskiy,UZ,83.12,overcast clouds,40.4272,71.7189,Ikathouse
16,Bathsheba,BB,86.61,few clouds,13.2167,-59.5167,Atlantis Hotel
19,San Eduardo,PH,82.33,overcast clouds,12.1679,125.4503,Kirum-Kirum Resort and Hotel
22,Avarua,CK,78.85,light rain,-21.2078,-159.7750,Paradise Inn
30,Butaritari,KI,82.63,broken clouds,3.0707,172.7902,Isles Sunset Lodge
31,Sacramento,US,85.71,broken clouds,38.4666,-121.3177,Woodside Homes


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Weather Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))